# Piano Triads Wavset

### This notebook shows how to extract a triad, play it and visualize the notes on a piano keyboard.


Link to dataset -> https://www.kaggle.com/davidbroberts/piano-triads-wavset

- This dataset consists of 432 Wav files of piano triads across 6 octaves. 

- There are wavs for each of the 12 major, minor and diminished triad chords in both the root and first inversion positions.

- The samples are 32 bit 44KHz mono, around 520K in size and 3 seconds in length. The total size of the wavs is around 220MB.

- The chords are whole note in duration with a tempo of 120.

- The CSV file (triads.csv) contains a list of each of the chord names, their octave on the keyboard, the inversion position, and the notes that make up the chord .

- The chord names are delimited by two underscore characters which split it into four parts. Chords are maj, min or dim.

- A lower case 's' following a note name denotes <i>sharp</i> notes/chords. Lower case b indicates <i>flat</i> notes/chords.

- "Cs_maj_2_0" indicates the chord is a C Sharp Major, second octave, root position (inversion).

- The octaves range from 2-7 .. with octave 5 being "middle C".

- Side note .. I used E flat instead of D sharp (for all the purists :).

- The Note1 column indicates the root note and its position. The remaining notes are in Note2 and Note3 columns.

- All audio samples were recorded on a DAW.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io.wavfile import read
from scipy import signal
from IPython.display import Audio, Image
import librosa
import librosa.display
import pylab
from numpy.fft import fft, ifft
%matplotlib inline
import math
from PIL import Image, ImageDraw

In [ ]:
# Load the data and get an idea of how the note names are stored
triads = pd.read_csv("/kaggle/input/piano-triads-wavset/triads.csv")
triads.head(20)

In [ ]:
# This is a function to build a piano keyboard
#
def make_keyboard(chord):
    key_width = 15
    key_height = 100
    octaves = 7
    
    white_keys = ["C","D","E","F","G","A","B"]
    black_keys = ['Cs',"Eb","Fs","Gs","Bb"]
    
    notes2 = []
    notes2.append([str(chord['Note1'].values[0]).split("_")[0], str(chord['Note1'].values[0]).split("_")[1]])
    notes2.append([str(chord['Note2'].values[0]).split("_")[0], str(chord['Note2'].values[0]).split("_")[1]])
    notes2.append([str(chord['Note3'].values[0]).split("_")[0], str(chord['Note3'].values[0]).split("_")[1]])
    
    img_width = (key_width) * 7 * (octaves + 1)
    img_height = key_height + 5

    img = Image.new("RGB", (img_width, img_height), color=(230,230,230))
    img1 = ImageDraw.Draw(img)

    # Draw the white keys
    for i in range(2, octaves + 2):
        offset = (i * (key_width + 2) * 7) - ((key_width + 2) * 7 * 2)
        
        # Draw the white keys
        j = 1
        for note in white_keys:
            fill_color = "FFFFFF"
            if (note == notes2[0][0] and str(i) == notes2[0][1]) or (note == notes2[1][0] and str(i) == notes2[1][1]) or (note == notes2[2][0] and str(i) == notes2[2][1]):
                fill_color = "FF0000"
            x = j * (key_width + 2) + offset - key_width
            w = j * (key_width + 2) + key_width + offset - key_width
            img1.rectangle([(x, 0), (w, key_height)], fill = "#" + fill_color, outline ="#555555")
            j += 1

    # Draw the black keys
    for i in range(2, octaves + 2):
        offset = (i * (key_width + 2) * 7) - ((key_width + 2) * 7 * 2)
        j = 1
        for note in black_keys:
            fill_color = "444444"
            if (note == notes2[0][0] and str(i) == notes2[0][1]) or (note == notes2[1][0] and str(i) == notes2[1][1]) or (note == notes2[2][0] and str(i) == notes2[2][1]):
                fill_color = "FF0000"

            if (j == 3):
                j += 1   
            x = j * (key_width + 2) + offset - key_width + (key_width / 2) + 4
            w = j * (key_width + 2) + key_width + offset - key_width  + (key_width / 2)
            img1.rectangle([(x , 0), (w, key_height * .65)], fill = "#" + fill_color, outline = "#555555")
                
            j += 1

    display(img)

- This is where we specify the chord, it's position on the keyboard, and it's inversion.
- Chord names follow this convention.
-- Gs_maj = G Sharp Major
-- A_min = A Minor
-- Eb_maj = E Flat Major
-- D_dim = D diminished


- Octaves are from 2-7.

- Inversion can be 0 or 1. 0 = Root position, 1 = First Inversion

In [ ]:
# The chord we want to get (all chord names are uppercase, maj/min is in lowercase)
chord = "G_dim"

# The Octave
octave = "6"

# Inversion (Root = 0, First Inversion = 1)
inversion = "0"

# Load the wav and make a player
filename = chord + "_" + octave + "_" + inversion

In [ ]:
# Get the note names/positions from the data
chord = triads[triads['Chord'] == filename]

# Display the notes on the keyboard
make_keyboard(chord)

In [ ]:
# Read the wav with IPython and make a player
sample_rate, wav_data = read("../input/piano-triads-wavset/piano_triads/" + filename + ".wav")
Audio(wav_data, rate=sample_rate)

In [ ]:
# Plot out some waveforms and a Mel Freq spectrogram
y, sr = librosa.load("../input/piano-triads-wavset/piano_triads/" + filename + ".wav")
y_harm, y_perc = librosa.effects.hpss(wav_data)

fig, ax = plt.subplots(nrows=1, ncols= 3, figsize=(15,5))
librosa.display.waveplot(y, sr=sr, ax=ax[0])

librosa.display.waveplot(y_harm, sr=sr, alpha=0.25, ax=ax[1])
librosa.display.waveplot(y_perc, sr=sr, color='r', alpha=0.5, ax=ax[1])

S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
S_dB = librosa.power_to_db(S, ref=np.max)
mel = librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000, ax=ax[2])
fig.colorbar(mel, ax=ax[2], format='%+2.0f dB')

ax[0].set(title='Monophonic');
ax[1].set(title='Harmonic + Percussive');
ax[2].set(title='Mel-frequency spectrogram');
ax[2].set(title='New');

In [ ]:
n_fft = 2048
hop_length = 512

stft = librosa.core.stft(wav_data, hop_length=hop_length, n_fft=n_fft)
spectrogram = np.abs(stft)

log_spectrogram = librosa.amplitude_to_db(spectrogram)
librosa.display.specshow(log_spectrogram, sr=sr, hop_length=hop_length)
plt.show()